In [ ]:
import dataclasses
import json
from pathlib import Path
from random import shuffle

import pandas as pd
from ipywidgets import Output
from line_profiler import profile

%load_ext line_profiler


In [ ]:
with Path("./words_dictionary.json").open() as f:
    words = list(set(json.load(f).keys()))

shuffle(words)
words.sort()
i = 0
def next_word() -> str:
    """return one word each call, never repeat the same word"""
    global i
    i += 1
    assert i < len(words), "No more words"
    return words[i-1]

In [ ]:
@dataclasses.dataclass(order=True, frozen=True)
class SortKey:
    key: int
    generation: int

@dataclasses.dataclass
class OrderableListItem:
    sort_key: SortKey
    value: str

class OrderableList:
    """test example of a list that can be ordered
    goint to benchmark the how fast the key lenth grows with the number of
    insertions and reorders
    """
    def __init__(self, keysize_bytes: int = 1):
        MAX = 2 ** (keysize_bytes * 8) - 1
        assert MAX & (MAX + 1) == 0, f"Max must be all 1s in binary: {MAX:b}"

        self.items: list[OrderableListItem] = []

        self.MIN: int = 0
        self.MAX: int = MAX

        self.historical_sort_keys: list[SortKey] = []


    @property
    def sorted_items(self) -> list[OrderableListItem]:
        return sorted(self.items, key=lambda x: x.sort_key)

    @profile
    def get_key(self, after: SortKey, before: SortKey) -> SortKey:
        """Create sort key for a new item that should be inserted between"""

        assert after != before, f"You can never find a gap between a key and itself: {after}."
        assert after.generation == before.generation, f"Can't compare keys from different generations: {after} and {before}"

        if before.key - after.key == 1:
            # print(f"No gap between {after} and {before}")
            after, before = self.rebalance(after, before)

        if before.key - after.key == 1:
            # print(len(self.items), "items stored")
            # print("All keys:")
            # print('hex:', ', '.join(f'{i.sort_key.key:x}' for i in self.sorted_items))
            # print('dec:', ', '.join(f'{i.sort_key.key}' for i in self.sorted_items))
            # print('bin:', ', '.join(f'{i.sort_key.key:b}' for i in self.sorted_items))
            raise ValueError(f"Can't find a gap between {after} and {before}, even after rebalancing, sort keys are exhausted.")

        gap = (before.key - after.key) // 2
        key = after.key + gap
        return SortKey(key, after.generation)

    @profile
    def append(self, v: str) -> None:
        """Append an item to the end of the list"""
        after = SortKey(self.MIN, 0) if len(self.items) == 0 else self.items[-1].sort_key
        key = self.get_key(after, SortKey(self.MAX, after.generation))
        oli = OrderableListItem(key, v)
        # print(oli)
        self.items.append(oli)

    @profile
    def rebalance(self, after: SortKey, before: SortKey) -> tuple[SortKey, SortKey]:
        """Rebalance the sort keys
        Evenly distribute the sort keys starting from 0 to self.MAX

        Returns the new mapping for after and before
        """

        assert after.generation == before.generation, f"Can't compare keys from different generations: {after} and {before}"
        generation = after.generation + 1
        GAP = self.MAX // len(self.items)

        new_after, new_before = SortKey(after.key, generation), SortKey(before.key, generation)

        for i, oli in enumerate(self.sorted_items):
            new_key = SortKey(i * GAP, generation)

            # print(f"Rebalancing {oli.sort_key} to {new_key}")
            # print(f"after: {after}, before: {before}")

            if oli.sort_key == after:
                new_after = new_key
            if oli.sort_key == before:
                new_before = new_key

            oli.sort_key = new_key

        return new_after, new_before

    ##################################

    def sanity_check_insertion_order(self) -> None:
        """check that the items are in order after intial insertions"""
        from itertools import pairwise
        for a, b in pairwise(self.sorted_items):
            if a.value > b.value:
                raise ValueError(f"Items are out of order: {a} > {b}")

    def sanity_check_key_uniqueness(self) -> None:
        """check that the keys are unique"""
        keys = [i.sort_key for i in self.sorted_items]
        if len(keys) != len(set(keys)):
            raise ValueError("Keys are not unique")



In [ ]:
ol = OrderableList(8)
out = Output()

@out.capture(clear_output=True, wait=True)
def showit() -> None:
    display(pd.DataFrame(ol.sorted_items))

def doit() -> None:
    for _ in range(10000):
        ol.append(next_word())
out

In [ ]:
%lprun -f ol.rebalance -f ol.get_key -f ol.append doit()